In [4]:
!pip install -q tensorflow-text

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\users\\andre\\anaconda3\\envs\\thesis_downgrade\\lib\\site-packages\\~rpc\\_cython\\cygrpc.cp36-win_amd64.pyd'
Consider using the `--user` option or check the permissions.

